In [1]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

### Define the functions

In [ ]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
import json
def getCurrentWeather(location:str, unit:str="fahrenheit")->str | dict | None:
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "los angeles" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

def getNickname(location:str)->str:
    """Get the nickname of a city"""
    if "tokyo" in location.lower():
        return "tk"
    elif "los angeles" in location.lower():
        return "la"
    elif "paris" in location.lower():
        return "py"
    else:
        return location
    



In [ ]:
import json

def show_json(message, obj):
    display(message, json.loads(obj.model_dump_json()))



### Create assistant

In [ ]:
from openai.types.beta.assistant import Assistant

assistant: Assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-3.5-turbo-1106",
  tools=[{
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  }, {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    } 
  }]
)

### Create Thread

In [ ]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_EWQe1GNT89NfAKNtVtM5F3zX', created_at=1702313355, metadata={}, object='thread')


### Add a message to thread

In [ ]:
from openai.types.beta.threads.thread_message import ThreadMessage

# First Request
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="How is the weather in Los Angles?"
)


In [ ]:
dict(message)

{'id': 'msg_gHQp6CgRNolShZuTnmVjrbGh',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='How is the weather in Los Angles?'), type='text')],
 'created_at': 1702313356,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_EWQe1GNT89NfAKNtVtM5F3zX'}

### Run the Assistant

In [ ]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

dict(run)


{'id': 'run_xxVdNEe4lYNqHIlzhgS34whN',
 'assistant_id': 'asst_MFo5G2UL16bU9tV1riAXYdGM',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702313357,
 'expires_at': 1702313957,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a weather bot. Use the provided functions to answer questions.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_EWQe1GNT89NfAKNtVtM5F3zX',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='getCurrentWeather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}, description='Get the weather in location'), type='function'),
  ToolAssistantToolsFunction(function=FunctionDefinition(name='getNickname', parameters={'type': 'object

In [ ]:
available_functions = {
    "getCurrentWeather": getCurrentWeather,
    "getNickname": getNickname
}

### Polling for Updates and Calling Functions


In [ ]:
import time

  # Loop until the run completes or requires action
while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,
                                                  run_id=run.id)
    # Add run steps retrieval here for debuging
    run_steps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    show_json("Run Steps-1:", run_steps)
    print("---------above runstep---------")
    print("runStatus.status=2",  runStatus.status ,'.....')

    # This means run is making a function call   
    if runStatus.status == "requires_action":
        print(runStatus.status ,'3.....')
        print("Status: 4 ", "requires_action")
        show_json("submit_tool_outputs", runStatus.required_action)
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls present:5")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls
            print("toolcalls-----", toolcall)

            tool_outputs = []
            for toolcall in toolCalls:
                function_name = toolcall.function.name
                function_args = json.loads(toolcall.function.arguments)
                
                if function_name in available_functions:
                    
                    
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="getCurrentWeather"," 6================================================================")
                  
                    if function_to_call.__name__ == "getCurrentWeather":
                        
                        response = function_to_call(
                        location=function_args.get("location"),
                        unit=function_args.get("unit")
                        )
                        
                        
                        tool_outputs.append({
                                  "tool_call_id": toolcall.id,
                                  "output": response
                              })
                    
                    elif function_to_call.__name__ == "getNickname":
                        response = function_to_call(
                          location=function_args.get("location")
                          )
                        tool_outputs.append({
                          "tool_call_id": toolcall.id,
                          "output": response,
                              })
            print(tool_outputs,"7--->>>>>") 
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
      
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........8--logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break

c:\Users\hamza\anaconda3\envs\genai\Lib\site-packages\pydantic\main.py:352: UserWarning: Pydantic serializer warnings:
  Expected `Union[MessageCreationStepDetails, ToolCallsStepDetails]` but got `MessageCreationStepDetails` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


'Run Steps-1:'

{'data': [{'id': 'step_6qHFnAmKv1BcUe8pvTpa5fo6',
   'assistant_id': 'asst_MFo5G2UL16bU9tV1riAXYdGM',
   'cancelled_at': None,
   'completed_at': None,
   'created_at': 1702313358,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_xxVdNEe4lYNqHIlzhgS34whN',
   'status': 'in_progress',
   'step_details': {'message_creation': None,
    'type': 'tool_calls',
    'tool_calls': [{'id': 'call_KnWQkZdjCaemPHhRuzgKFLmi',
      'type': 'function',
      'function': {'name': 'getCurrentWeather',
       'arguments': '{"location":"Los Angeles, CA","unit":"c"}'}}]},
   'thread_id': 'thread_EWQe1GNT89NfAKNtVtM5F3zX',
   'type': 'tool_calls',
   'expires_at': 1702313957}],
 'object': 'list',
 'first_id': 'step_6qHFnAmKv1BcUe8pvTpa5fo6',
 'last_id': 'step_6qHFnAmKv1BcUe8pvTpa5fo6',
 'has_more': False}

---------above runstep---------
runStatus.status=2 requires_action .....
requires_action 3.....
Status: 4  requires_action


'submit_tool_outputs'

...

{'data': [{'id': 'step_6qHFnAmKv1BcUe8pvTpa5fo6',
   'assistant_id': 'asst_MFo5G2UL16bU9tV1riAXYdGM',
   'cancelled_at': None,
   'completed_at': 1702313667,
   'created_at': 1702313358,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_xxVdNEe4lYNqHIlzhgS34whN',
   'status': 'completed',
   'step_details': {'tool_calls': [{'id': 'call_KnWQkZdjCaemPHhRuzgKFLmi',
      'function': {'arguments': '{"location":"Los Angeles, CA","unit":"c"}',
       'name': 'getCurrentWeather',
       'output': '{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'},
      'type': 'function'}],
    'type': 'tool_calls'},
   'thread_id': 'thread_EWQe1GNT89NfAKNtVtM5F3zX',
   'type': 'tool_calls',
   'expires_at': 1702313957}],
 'object': 'list',
 'first_id': 'step_6qHFnAmKv1BcUe8pvTpa5fo6',
 'last_id': 'step_6qHFnAmKv1BcUe8pvTpa5fo6',
 'has_more': False}

---------above runstep---------
runStatus.status=2 in_progress .....
Run is queued. Waiting...


'Run Steps-1:'

{'data': [{'id': 'step_WqWycIvRD3ZyITKG1zALJrKp',
   'assistant_id': 'asst_MFo5G2UL16bU9tV1riAXYdGM',
   'cancelled_at': None,
   'completed_at': 1702313669,
   'created_at': 1702313669,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_xxVdNEe4lYNqHIlzhgS34whN',
   'status': 'completed',
   'step_details': {'message_creation': {'message_id': 'msg_E6HYlweBhw88U98Fw4GdHPPx'},
    'type': 'message_creation'},
   'thread_id': 'thread_EWQe1GNT89NfAKNtVtM5F3zX',
   'type': 'message_creation',
   'expires_at': None},
  {'id': 'step_6qHFnAmKv1BcUe8pvTpa5fo6',
   'assistant_id': 'asst_MFo5G2UL16bU9tV1riAXYdGM',
   'cancelled_at': None,
   'completed_at': 1702313667,
   'created_at': 1702313358,
   'expired_at': None,
   'failed_at': None,
   'last_error': None,
   'metadata': None,
   'object': 'thread.run.step',
   'run_id': 'run_xxVdNEe4lYNqHIlzhgS34whN',
   'status': 'completed',
   'step_details': {'to

---------above runstep---------
runStatus.status=2 completed .....
completed...........8--logic
Assistant: The current weather in Los Angeles is 72°F.

User: How is the weather in Los Angles?

